## 0.1 Setup

In [15]:
!pip install biopython


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.1 MB/s eta 0:00:00a 0:00:01


In [4]:
!pwd

/home/osheakes/Research_Project_MMM/Fasta


## 

# 1.0 Viral Precursor Blast 

## 1.1 Viral Blast test

In [ ]:
#test run of one Accession to ensure command runs as expected
!blastp -remote -query input_proteins.fa -db genomic/Viruses/Viral_Protein_Sequences -evalue 0.06 -outfmt 7 -out output.txt

# 

## 1.2 Blast in fmt 5 format

In [2]:
!mkdir /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/

In [1]:
import os
import time

# Record start time
start_time = time.time()

# fasta inputs
fasta_directory = "/home/osheakes/Research_Project_MMM/Fasta"

# Directory for results
blast_output_directory = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/"

# BLAST parameters
blast_db = "genomic/Viruses/Viral_Protein_Sequences"
evalue = "0.06"
outfmt = "5"  
blast_remote = "-remote"  

# 5min interval for logbook
print_interval = 300 

# Set initial time for printing logbook
next_print_time = start_time + print_interval

# Select fasta's in directory
for filename in os.listdir(fasta_directory):
    if filename.endswith(".fasta"):  
        fasta_file = os.path.join(fasta_directory, filename)
        output_file = os.path.splitext(filename)[0] + "_blast_results.txt"
        output_path = os.path.join(blast_output_directory, output_file)
        
        # Check if output file already exists
        if os.path.exists(output_path):
            print(f"Skipping {filename} - Output file already exists: {output_path}")
            continue
        
        # Record start time for each file
        file_start_time = time.time()
        
        # BLAST command 
        command = f"blastp {blast_remote} -query {fasta_file} -db {blast_db} -evalue {evalue} -outfmt {outfmt} -out {output_path}"
        
        # Execute BLAST command
        os.system(command)
        
        # Calculate time taken for BLAST search on current file
        file_duration = time.time() - file_start_time
        
        #  to print logbook
        if time.time() >= next_print_time:
            # Print logbook message
            print(f"BLAST for {filename} started at {time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(file_start_time))} and completed in {file_duration//60:.0f} minutes {file_duration%60:.2f} seconds. Results saved in {output_path}")
            
            # Update next print time
            next_print_time += print_interval
        
# Calculate total run time
total_duration = time.time() - start_time

# Print total execution time
print(f"Total execution time: {total_duration//60:.0f} minutes {total_duration%60:.2f} seconds")


Skipping O96011.fasta - Output file already exists: /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/O96011_blast_results.txt
Skipping Q07812.fasta - Output file already exists: /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/Q07812_blast_results.txt
Skipping Q8NFJ6.fasta - Output file already exists: /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/Q8NFJ6_blast_results.txt
Skipping P07492.fasta - Output file already exists: /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/P07492_blast_results.txt
Skipping Q15303.fasta - Output file already exists: /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/Q15303_blast_results.txt
Skipping Q15726.fasta - Output file already exists: /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/Q15726_blast_results.txt
Skipping O00910.fasta - Output file already exists: /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/O00910_blast_results.txt
Skipping P38398.fasta - Output file alrea

In [8]:
import os

# Path to the directory containing the FASTA files
directory_path = "/home/osheakes/Research_Project_MMM/Fasta/"

# Count the number of FASTA files in the directory
fasta_files = [file for file in os.listdir(directory_path) if file.endswith(".fasta")]
num_fasta_files = len(fasta_files)

print(f"Number of FASTA files in directory '{directory_path}': {num_fasta_files}")


Number of FASTA files in directory '/home/osheakes/Research_Project_MMM/Fasta/': 119


## 1.3 Create script to parse data out of blast results files

In [2]:
import os
import re
import csv

# Path to the blast outputs in fmt 5
results_directory = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5"

# Output CSV file path
output_csv_path = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/VB_Results.csv"

# Initialize count variables
included_files_count = 0
ignored_files_count = 0

# Write CSV header
with open(output_csv_path, "w", newline="") as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["Accession ID", "Hit Number", "Hit ID", "Hit Length", "Species", "Query Definition", "Hsp Bit Score", "Hsp Score", "Hsp E-value", "Identity", "Positives", "Gaps", "Hsp Qseq", "Hsp Hseq"])

# Iterate through each blast output
for filename in os.listdir(results_directory):
    if filename.endswith(".txt") and filename != ".txt":  
        print(f"Processing file: {filename}")
        results_file_path = os.path.join(results_directory, filename)
        
        # Open and read the results
        with open(results_file_path, "r") as results_file:
            results_text = results_file.read()

            # Check if "0 hits found" or "No hits found" is present
            if "0 hits found" in results_text or "No hits found" in results_text:
                print(f"Ignoring file: {filename} - No hits found")
                ignored_files_count += 1
                continue
            
            # Check if the file contains expected XML structure
            if "<Hit>" not in results_text:
                print(f"Ignoring file: {filename} - Unexpected format")
                ignored_files_count += 1
                continue
            
            # Increment included files count
            included_files_count += 1
            
            # Use file names to get Accession ID's
            accession_id = os.path.splitext(filename)[0].replace("_blast_results", "")
            
            # Initialize hit number
            hit_number = 1
            
            # Specify Hit ID
            hit_ids = re.findall(r'<Hit_id>.*?\|(.*?)\|</Hit_id>', results_text)
            
            # Specify hit length
            hit_lengths = re.findall(r'<Hit_len>(\d+)</Hit_len>', results_text)
            
            # Specify species
            species = re.findall(r'\[(.*?)\]', results_text)

            # Specify Query Definition
            query_definitions = re.findall(r'<Hit_def>(.*?)<', results_text)

            # Specify additional Hsp information
            hsp_bit_scores = re.findall(r'<Hsp_bit-score>(.*?)</Hsp_bit-score>', results_text)
            hsp_scores = re.findall(r'<Hsp_score>(.*?)</Hsp_score>', results_text)
            hsp_evals = re.findall(r'<Hsp_evalue>(.*?)</Hsp_evalue>', results_text)
            hsp_qseqs = re.findall(r'<Hsp_qseq>(.*?)</Hsp_qseq>', results_text)
            hsp_hseqs = re.findall(r'<Hsp_hseq>(.*?)</Hsp_hseq>', results_text)
            hsp_identities = re.findall(r'<Hsp_identity>(.*?)</Hsp_identity>', results_text)
            hsp_positives = re.findall(r'<Hsp_positive>(.*?)</Hsp_positive>', results_text)
            hsp_gaps = re.findall(r'<Hsp_gaps>(.*?)</Hsp_gaps>', results_text)

            # Write to CSV
            with open(output_csv_path, "a", newline="") as csvfile:
                csv_writer = csv.writer(csvfile)
                for hit_id, length, word, query_def, bit_score, score, eval, identity, positives, gaps, qseq, hseq in zip(hit_ids, hit_lengths, species, query_definitions, hsp_bit_scores, hsp_scores, hsp_evals, hsp_identities, hsp_positives, hsp_gaps, hsp_qseqs, hsp_hseqs):
                    if query_def:  # Check if query definition is not empty
                        csv_writer.writerow([accession_id, f"Hit {hit_number}", hit_id, length, word, query_def, bit_score, score, eval, identity, positives, gaps, qseq, hseq])
                        hit_number += 1

# Print count of included and ignored files
print(f"Included files: {included_files_count}")
print(f"Ignored files: {ignored_files_count}")
print("CSV generation complete.")


Processing file: P12872_blast_results.txt
Ignoring file: P12872_blast_results.txt - No hits found
Processing file: P56199_blast_results.txt
Processing file: P81605_blast_results.txt
Ignoring file: P81605_blast_results.txt - No hits found
Processing file: P01579_blast_results.txt
Ignoring file: P01579_blast_results.txt - No hits found
Processing file: P18509_blast_results.txt
Ignoring file: P18509_blast_results.txt - No hits found
Processing file: P01258_blast_results.txt
Ignoring file: P01258_blast_results.txt - No hits found
Processing file: P01570_blast_results.txt
Ignoring file: P01570_blast_results.txt - No hits found
Processing file: P23582_blast_results.txt
Ignoring file: P23582_blast_results.txt - No hits found
Processing file: P24941_blast_results.txt
Processing file: P19838_blast_results.txt
Processing file: O95390_blast_results.txt
Processing file: Q9HC23_blast_results.txt
Ignoring file: Q9HC23_blast_results.txt - No hits found
Processing file: Q8NFJ6_blast_results.txt
Proces

In [26]:
import os
import csv

# Path to the blast outputs in fmt 5
results_directory = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5"

# Output CSV file path for no-hit queries
no_hits_csv_path = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/No_Hits_Results.csv"

# Initialize count variables
ignored_files_count = 0

# Write CSV header for no-hit queries
with open(no_hits_csv_path, "w", newline="") as no_hits_file:
    no_hits_writer = csv.writer(no_hits_file)
    no_hits_writer.writerow(["Filename", "Reason"])

# Iterate through each blast output
for filename in os.listdir(results_directory):
    if filename.endswith(".txt") and filename != ".txt":
        print(f"Processing file: {filename}")
        results_file_path = os.path.join(results_directory, filename)
        
        # Open and read the results
        with open(results_file_path, "r") as results_file:
            results_text = results_file.read()

            # Check if "0 hits found" or "No hits found" is present
            if "0 hits found" in results_text or "No hits found" in results_text:
                print(f"Ignoring file: {filename} - No hits found")
                ignored_files_count += 1
                with open(no_hits_csv_path, "a", newline="") as no_hits_file:
                    no_hits_writer = csv.writer(no_hits_file)
                    no_hits_writer.writerow([filename, "No hits found"])
                continue
            
            # Check if the file contains expected XML structure
            if "<Hit>" not in results_text:
                print(f"Ignoring file: {filename} - Unexpected format")
                ignored_files_count += 1
                with open(no_hits_csv_path, "a", newline="") as no_hits_file:
                    no_hits_writer = csv.writer(no_hits_file)
                    no_hits_writer.writerow([filename, "Unexpected format"])
                continue

# Print count of ignored files
print(f"Ignored files: {ignored_files_count}")
print("CSV generation for no-hit queries complete.")

Processing file: P12872_blast_results.txt
Ignoring file: P12872_blast_results.txt - No hits found
Processing file: P56199_blast_results.txt
Processing file: P81605_blast_results.txt
Ignoring file: P81605_blast_results.txt - No hits found
Processing file: P01579_blast_results.txt
Ignoring file: P01579_blast_results.txt - No hits found
Processing file: P18509_blast_results.txt
Ignoring file: P18509_blast_results.txt - No hits found
Processing file: P01258_blast_results.txt
Ignoring file: P01258_blast_results.txt - No hits found
Processing file: P01570_blast_results.txt
Ignoring file: P01570_blast_results.txt - No hits found
Processing file: P23582_blast_results.txt
Ignoring file: P23582_blast_results.txt - No hits found
Processing file: P24941_blast_results.txt
Processing file: P19838_blast_results.txt
Processing file: O95390_blast_results.txt
Processing file: Q9HC23_blast_results.txt
Ignoring file: Q9HC23_blast_results.txt - No hits found
Processing file: Q8NFJ6_blast_results.txt
Proces

## Process files which produced No hits

In [29]:
import os
import re
import shutil

# Paths
fasta_directory = "/home/osheakes/Research_Project_MMM/Fasta"
blast_output_directory = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/"
no_hits_directory = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/no_hits"

# Create an output directory for files which produced no hits
os.makedirs(no_hits_directory, exist_ok=True)

# Initialize count variables
ignored_files_count = 0

# Iterate through each BLAST output
for filename in os.listdir(blast_output_directory):
    if filename.endswith(".txt") and filename != ".txt":
        results_file_path = os.path.join(blast_output_directory, filename)
        
        # Open and read the results
        with open(results_file_path, "r") as results_file:
            results_text = results_file.read()
            
            # Check if "0 hits found" or "No hits found" is present
            if "0 hits found" in results_text or "No hits found" in results_text:
                ignored_files_count += 1
                # Determine the original fasta file
                fasta_filename = filename.replace("_blast_results.txt", ".fasta")
                fasta_file_path = os.path.join(fasta_directory, fasta_filename)
                
                # Move the original fasta file to the no_hits directory
                if os.path.exists(fasta_file_path):
                    shutil.move(fasta_file_path, os.path.join(no_hits_directory, fasta_filename))
                    print(f"Moved no-hit file: {fasta_filename} to {no_hits_directory}")

print(f"Ignored (no-hit) files: {ignored_files_count}")
print("No-hit files moved to the designated directory.")


Moved no-hit file: P12872.fasta to /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/no_hits
Moved no-hit file: P81605.fasta to /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/no_hits
Moved no-hit file: P01579.fasta to /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/no_hits
Moved no-hit file: P18509.fasta to /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/no_hits
Moved no-hit file: P01258.fasta to /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/no_hits
Moved no-hit file: P01570.fasta to /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/no_hits
Moved no-hit file: P23582.fasta to /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/no_hits
Moved no-hit file: Q9HC23.fasta to /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/no_hits
Moved no-hit file: P20366.fasta to /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/no_hits
Moved no-hit file: P01568.fasta to /home/osheakes/Research_Project_MMM/Fa

# 

# 

# 

# 

# 

# 

## Check NCBI Viral Non-Redundancies Database for hits

In [30]:
import os
import subprocess
import time

# Paths
no_hits_directory = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/no_hits"
blast_output_directory = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/nr_blastp_results"
os.makedirs(blast_output_directory, exist_ok=True)

# BLAST parameters
db = "nr"
evalue = "0.06"
remote = "-remote"
entrez_query = '"viruses[orgn]"' 

# Record the start time
start_time = time.time()
start_time_local = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(start_time))
print(f"Start time (Local): {start_time_local}")

# Iterate through the FASTA files in the no_hits directory
for filename in os.listdir(no_hits_directory):
    if filename.endswith(".fasta"):
        fasta_file = os.path.join(no_hits_directory, filename)
        output_file = os.path.splitext(filename)[0] + "_blastp_results.txt"
        output_path = os.path.join(blast_output_directory, output_file)
        
        # Define the BLAST command
        command = f"blastp -db {db} -query {fasta_file} -out {output_path} -evalue {evalue} {remote} -entrez_query {entrez_query}"
        
        # Execute the BLAST command
        try:
            result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            if result.returncode == 0:
                print(f"BLAST command executed successfully for {filename}.")
            else:
                print(f"BLAST command failed for {filename} with return code {result.returncode}.")
        except subprocess.CalledProcessError as e:
            print(f"BLAST command failed for {filename} with error: {e.stderr.decode()}")
        
        # Record the end time
        end_time = time.time()
        end_time_local = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(end_time))

        # Calculate the duration
        duration = end_time - start_time
        duration_minutes, duration_seconds = divmod(duration, 60)

        print(f"End time (Local): {end_time_local}")
        print(f"Execution time for {filename}: {int(duration_minutes)} minutes and {duration_seconds:.2f} seconds")

# Calculate total run time
total_duration = time.time() - start_time
total_duration_minutes, total_duration_seconds = divmod(total_duration, 60)

print(f"Total execution time: {int(total_duration_minutes)} minutes and {total_duration_seconds:.2f} seconds")


Start time (Local): 2024-06-04 12:33:52
BLAST command executed successfully for O96011.fasta.
End time (Local): 2024-06-04 13:00:24
Execution time for O96011.fasta: 26 minutes and 31.48 seconds
BLAST command executed successfully for P07492.fasta.
End time (Local): 2024-06-04 13:31:56
Execution time for P07492.fasta: 58 minutes and 3.42 seconds
BLAST command executed successfully for Q15726.fasta.
End time (Local): 2024-06-04 14:48:39
Execution time for Q15726.fasta: 134 minutes and 46.41 seconds
BLAST command executed successfully for P06881.fasta.
End time (Local): 2024-06-04 15:15:10
Execution time for P06881.fasta: 161 minutes and 17.44 seconds
BLAST command executed successfully for P20366.fasta.
End time (Local): 2024-06-04 15:41:41
Execution time for P20366.fasta: 187 minutes and 49.20 seconds
BLAST command executed successfully for P35318.fasta.
End time (Local): 2024-06-04 16:13:14
Execution time for P35318.fasta: 219 minutes and 21.42 seconds
BLAST command executed successful

In [10]:
import os
import subprocess
import time
from datetime import datetime, timezone, timedelta

# Function to convert GMT to Dublin local time
def gmt_to_dublin(gmt_time):
    dublin_offset = timedelta(hours=1)
    return gmt_time + dublin_offset

# Paths
fasta_directory = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/no_hits"
blast_output_directory = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/nr_blastp_results"

# Create the blast output directory if it doesn't exist
os.makedirs(blast_output_directory, exist_ok=True)

# Record the start time of the entire process in GMT
start_time = datetime.now(timezone.utc)
start_time_dublin = gmt_to_dublin(start_time)
print(f"Process started at: {start_time_dublin.strftime('%Y-%m-%d %H:%M:%S')} Dublin local time")

def get_accession_id(fasta_file):
    with open(fasta_file, 'r') as file:
        for line in file:
            if line.startswith('>'):
                return line.split()[0][1:]
    return None

# Initialize the file counters
num_files_processed = 0
num_files_skipped = 0

# Iterate through each fasta file in the directory
for filename in os.listdir(fasta_directory):
    if filename.endswith(".fasta"):
        fasta_file = os.path.join(fasta_directory, filename)
        accession_id = get_accession_id(fasta_file)
        if not accession_id:
            print(f"No accession ID found in {filename}, skipping...")
            num_files_skipped += 1
            continue
        
        output_file = os.path.join(blast_output_directory, f"{filename.split('.')[0]}_blastp_results.txt")
        
        # Skip if output file already exists
        if os.path.exists(output_file):
            print(f"Skipping {filename} as output file {output_file} already exists.")
            num_files_skipped += 1
            continue
        
        # Construct the blastp command for remote search
        blastp_command = [
            "blastp",
            "-query", fasta_file,
            "-db", "nr",
            "-out", output_file,
            "-evalue", "0.06",
            "-entrez_query", "viruses[orgn]",
            "-remote"
        ]
        
        # Record start time for this file
        file_start_time = time.time()
        
        # Execute the blastp command
        print(f"Running blast search for {filename} (Accession ID: {accession_id})...")
        try:
            result = subprocess.run(blastp_command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            if result.returncode == 0:
                print(f"BLAST command executed successfully for {filename}.")
            else:
                print(f"BLAST command failed for {filename} with return code {result.returncode}.")
        except subprocess.CalledProcessError as e:
            print(f"BLAST command failed for {filename} with error: {e.stderr.decode()}")
        
        # Calculate elapsed time for this file
        file_elapsed_time = time.time() - file_start_time
        
        # Print filename and elapsed time
        print(f"Blast search for {filename} completed in {file_elapsed_time:.2f} seconds.")
        
        # Increment the file counter
        num_files_processed += 1
        
        # Check if 30 minutes have elapsed since the start of the process
        if (datetime.now(timezone.utc) - start_time).seconds >= 1800:
            print(f"Process has been running for 30 minutes. Current time: {gmt_to_dublin(datetime.now(timezone.utc)).strftime('%Y-%m-%d %H:%M:%S')} Dublin local time")
            start_time = datetime.now(timezone.utc)

print(f"Blastp searches completed for all fasta files. Number of files processed: {num_files_processed}, Number of files skipped: {num_files_skipped}")

Process started at: 2024-07-06 18:02:17 Dublin local time
Skipping O96011.fasta as output file /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/nr_blastp_results/O96011_blastp_results.txt already exists.
Skipping P07492.fasta as output file /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/nr_blastp_results/P07492_blastp_results.txt already exists.
Skipping Q15726.fasta as output file /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/nr_blastp_results/Q15726_blastp_results.txt already exists.
Skipping P06881.fasta as output file /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/nr_blastp_results/P06881_blastp_results.txt already exists.
Skipping P20366.fasta as output file /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/nr_blastp_results/P20366_blastp_results.txt already exists.
Skipping P35318.fasta as output file /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/nr_blastp_results/P35318_blastp_results.txt already exists.
Skipping P12872.fasta as outpu

In [32]:
import os

# Directory containing blast result files
blast_output_directory = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/nr_blastp_results/"

# Name of the master file
master_filename = "Vir_nr_master_blaster.txt"

# Path to the master file
master_filepath = os.path.join(blast_output_directory, master_filename)

# Open the master file in append mode
with open(master_filepath, "a") as master_file:
    # Iterate through each file in the blast output directory
    for filename in os.listdir(blast_output_directory):
        # Check if the file is a blast result file
        if filename.endswith("_blastp_results.txt"):
            # Get the full path of the blast result file
            blast_result_filepath = os.path.join(blast_output_directory, filename)
            # Open the blast result file and read its content
            with open(blast_result_filepath, "r") as blast_result_file:
                # Read the content of the blast result file and write it to the master file
                master_file.write(blast_result_file.read())
                # Write a newline character to separate the content of each blast result file
                master_file.write("\n")

print("All blast_results.txt files have been concatenated into the master file:", master_filepath)

All blast_results.txt files have been concatenated into the master file: /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/nr_blastp_results/Vir_nr_master_blaster.txt


In [33]:
from collections import defaultdict
import csv

# Path to the master file
master_filepath = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/nr_blastp_results/Vir_nr_master_blaster.txt"

# Initialize a defaultdict to store query counts and hits
query_hits = defaultdict(list)

# Read the file and extract query IDs and hits
try:
    with open(master_filepath, "r") as master_file:
        current_query_id = None
        for line in master_file:
            if "|" in line: 
                current_query_id = line.split("|")[1].strip()  
            elif ">" in line:  
                if current_query_id:
                    query_hits[current_query_id].append(line.split(">")[1].strip())  

    # Calculate total hits for each accession ID
    total_hits = {query_id: len(hits) for query_id, hits in query_hits.items()}

    # Rank and print total hits
    print("Ranked total hits for each accession ID:")
    for rank, (query_id, hits) in enumerate(sorted(total_hits.items(), key=lambda x: x[1], reverse=True), start=1):
        print(f"{rank}. Accession ID: {query_id}, Total Hits: {hits}")

    # Write results to CSV file
    output_csv_path = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/nr_blastp_results/Vir_nr_hit.csv"
    with open(output_csv_path, mode='w', newline='') as csv_file:
        fieldnames = ['Query ID', 'Hits']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        
        writer.writeheader()
        for query_id, hits in query_hits.items():
            writer.writerow({'Query ID': query_id, 'Hits': ', '.join(hits)})
    print(f"Results saved to '{output_csv_path}'")
except FileNotFoundError:
    print(f"Error: File '{master_filepath}' not found.")
    exit(1)
except Exception as e:
    print(f"An error occurred: {e}")
    exit(1)

Ranked total hits for each accession ID:
1. Accession ID: P42574, Total Hits: 1
Results saved to '/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/nr_blastp_results/Vir_nr_hit.csv'


# 

# 

# 

# 

## Refseq Search

In [12]:
import os
import subprocess
import time
from datetime import datetime, timezone, timedelta

# Function to convert GMT to Dublin local time
def gmt_to_dublin(gmt_time):
    dublin_offset = timedelta(hours=1)  
    return gmt_time + dublin_offset

# Paths
fasta_directory = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/no_hits"
blast_output_directory = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/refseq_blastp_results/"

# Create the blast output directory if it doesn't exist
os.makedirs(blast_output_directory, exist_ok=True)

# Record the start time of the entire process in GMT
start_time = datetime.now(timezone.utc)
start_time_dublin = gmt_to_dublin(start_time)
print(f"Process started at: {start_time_dublin.strftime('%Y-%m-%d %H:%M:%S')} Dublin local time")

def get_accession_id(fasta_file):
    with open(fasta_file, 'r') as file:
        for line in file:
            if line.startswith('>'):
                return line.split()[0][1:]  
    return None

# Initialize the file counters
num_files_processed = 0
num_files_skipped = 0

# Iterate through each fasta file in the directory
for filename in os.listdir(fasta_directory):
    if filename.endswith(".fasta"):
        fasta_file = os.path.join(fasta_directory, filename)
        accession_id = get_accession_id(fasta_file)
        if not accession_id:
            print(f"No accession ID found in {filename}, skipping...")
            num_files_skipped += 1
            continue
        
        output_file = os.path.join(blast_output_directory, f"{filename.split('.')[0]}_blastp_results.txt")
        
        # Skip if output file already exists
        if os.path.exists(output_file):
            print(f"Skipping {filename} as output file {output_file} already exists.")
            num_files_skipped += 1
            continue
        
        # Construct the blastp command for remote search
        blastp_command = [
            "blastp",
            "-query", fasta_file,
            "-db", "refseq_protein",
            "-out", output_file,
            "-evalue", "0.06",
            "-entrez_query", "viruses[orgn]",
            "-remote"
        ]
        
        # Record start time for this file
        file_start_time = time.time()
        
        # Execute the blastp command
        print(f"Running blast search for {filename} (Accession ID: {accession_id})...")
        try:
            result = subprocess.run(blastp_command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            if result.returncode == 0:
                print(f"BLAST command executed successfully for {filename}.")
            else:
                print(f"BLAST command failed for {filename} with return code {result.returncode}.")
        except subprocess.CalledProcessError as e:
            print(f"BLAST command failed for {filename} with error: {e.stderr.decode()}")
        
        # Calculate elapsed time for this file
        file_elapsed_time = time.time() - file_start_time
        
        # Print filename and elapsed time
        print(f"Blast search for {filename} completed in {file_elapsed_time:.2f} seconds.")
        
        # Increment the file counter
        num_files_processed += 1
        
        # Check if 30 minutes have elapsed since the start of the process
        if (datetime.now(timezone.utc) - start_time).seconds >= 1800:
            print(f"Process has been running for 30 minutes. Current time: {gmt_to_dublin(datetime.now(timezone.utc)).strftime('%Y-%m-%d %H:%M:%S')} Dublin local time")
            start_time = datetime.now(timezone.utc)

print(f"Blastp searches completed for all fasta files. Number of files processed: {num_files_processed}, Number of files skipped: {num_files_skipped}")

Process started at: 2024-07-06 18:25:39 Dublin local time
Skipping O96011.fasta as output file /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/refseq_blastp_results/O96011_blastp_results.txt already exists.
Skipping P07492.fasta as output file /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/refseq_blastp_results/P07492_blastp_results.txt already exists.
Skipping Q15726.fasta as output file /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/refseq_blastp_results/Q15726_blastp_results.txt already exists.
Skipping P06881.fasta as output file /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/refseq_blastp_results/P06881_blastp_results.txt already exists.
Skipping P20366.fasta as output file /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/refseq_blastp_results/P20366_blastp_results.txt already exists.
Skipping P35318.fasta as output file /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/refseq_blastp_results/P35318_blastp_results.txt already exists.
Skippi

# 

# 

# 

In [23]:
!mv /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/refseq_blastp_results/concatenated_no_hits_blastp_results.txt /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/

## Combine results from Refseq Search 

In [24]:
import os

# Directory containing blast result files
blast_output_directory = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/refseq_blastp_results/"

# Name of the master file
master_filename = "Vir_refseq_master_blaster.txt"

# Path to the master file
master_filepath = os.path.join(blast_output_directory, master_filename)

# Open the master file in append mode
with open(master_filepath, "a") as master_file:
    # Iterate through each file in the blast output directory
    for filename in os.listdir(blast_output_directory):
        # Check if the file is a blast result file
        if filename.endswith("_blastp_results.txt"):
            # Get the full path of the blast result file
            blast_result_filepath = os.path.join(blast_output_directory, filename)
            # Open the blast result file and read its content
            with open(blast_result_filepath, "r") as blast_result_file:
                # Read the content of the blast result file and write it to the master file
                master_file.write(blast_result_file.read())
                # Write a newline character to separate the content of each blast result file
                master_file.write("\n")

print("All blast_results.txt files have been concatenated into the master file:", master_filepath)


All blast_results.txt files have been concatenated into the master file: /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/refseq_blastp_results/Vir_refseq_master_blaster.txt


## Processing

In [25]:
from collections import defaultdict
import csv

# Path to the master file
master_filepath = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/refseq_blastp_results/Vir_refseq_master_blaster.txt"

# Initialize a defaultdict to store query counts and hits
query_hits = defaultdict(list)

# Read the file and extract query IDs and hits
try:
    with open(master_filepath, "r") as master_file:
        current_query_id = None
        for line in master_file:
            if "|" in line: 
                current_query_id = line.split("|")[1].strip()  
            elif ">" in line:  
                if current_query_id:
                    query_hits[current_query_id].append(line.split(">")[1].strip())  

    # Calculate total hits for each accession ID
    total_hits = {query_id: len(hits) for query_id, hits in query_hits.items()}

    # Rank and print total hits
    print("Ranked total hits for each accession ID:")
    for rank, (query_id, hits) in enumerate(sorted(total_hits.items(), key=lambda x: x[1], reverse=True), start=1):
        print(f"{rank}. Accession ID: {query_id}, Total Hits: {hits}")

    # Write results to CSV file
    output_csv_path = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/refseq_blastp_results/Vir_refseq_hit.csv"
    with open(output_csv_path, mode='w', newline='') as csv_file:
        fieldnames = ['Query ID', 'Hits']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        
        writer.writeheader()
        for query_id, hits in query_hits.items():
            writer.writerow({'Query ID': query_id, 'Hits': ', '.join(hits)})
    print(f"Results saved to '{output_csv_path}'")
except FileNotFoundError:
    print(f"Error: File '{master_filepath}' not found.")
    exit(1)
except Exception as e:
    print(f"An error occurred: {e}")
    exit(1)



Ranked total hits for each accession ID:
1. Accession ID: P02533, Total Hits: 24
2. Accession ID: Q03252, Total Hits: 20
3. Accession ID: P42574, Total Hits: 12
4. Accession ID: Q99988, Total Hits: 4
5. Accession ID: P15502, Total Hits: 3
6. Accession ID: P01241, Total Hits: 3
7. Accession ID: Q96BS2, Total Hits: 3
8. Accession ID: P52630, Total Hits: 3
9. Accession ID: Q10589, Total Hits: 3
10. Accession ID: P10082, Total Hits: 3
11. Accession ID: P01570, Total Hits: 3
Results saved to '/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/refseq_blastp_results/Vir_refseq_hit.csv'


In [26]:
import csv

# Path to the input text file
input_file_path = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/refseq_blastp_results/Vir_refseq_master_blaster.txt"

# Path to the output CSV file
output_csv_path = '/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/refseq_blastp_results/Vir_refseq_hit.csv'

# Initialize lists to store accession IDs and hit names
accession_ids = []
hit_names = []

# Read the input file and extract accession IDs and hit names
try:
    with open(input_file_path, "r") as input_file:
        current_accession_id = None
        for line in input_file:
            if "|" in line:
                current_accession_id = line.split("|")[1].strip()
            elif line.startswith(">"):
                if current_accession_id:
                    accession_ids.append(current_accession_id)
                    hit_name = line.strip()[1:] 
                    hit_names.append(hit_name)
except FileNotFoundError:
    print(f"Error: File '{input_file_path}' not found.")
    exit(1)
except Exception as e:
    print(f"An error occurred: {e}")
    exit(1)

# Write accession IDs and hit names to CSV file
try:
    with open(output_csv_path, mode='w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["Accession ID", "Hit Name"])  
        for accession_id, hit_name in zip(accession_ids, hit_names):
            writer.writerow([accession_id, hit_name])
    print(f"Accession IDs and hit names saved to '{output_csv_path}'")
except Exception as e:
    print(f"An error occurred while writing to CSV: {e}")

Accession IDs and hit names saved to '/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/refseq_blastp_results/Vir_refseq_hit.csv'


In [28]:
import os
import csv

# Source directory containing blast result files with hits
source_dir =  "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/refseq_blastp_results/"

# Output CSV file path
output_csv = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/refseq_blastp_results/comp_Vir_refseq_data.csv"

# Function to extract query ID from file name
def extract_query_id(file_name):
    return file_name.replace("_blastp_results.txt", "")

# Parse out required data from blast reaults
def parse_blast_results(file_path):
    hits_data = []
    with open(file_path, 'r') as f:
        query_id = extract_query_id(os.path.basename(file_path))
        hit_number = 0 
        hit_id = None 
        species_id = None 
        seq_length = None  
        evalue = None 
        score = None  
        identities = None  
        identities_percent = None
        positives = None 
        positives_percent = None  
        gaps = None  
        gaps_percent = None  
        for line in f:
            line = line.strip()
            if line.startswith(">"):
                hit_number += 1 
                hit_id = line.split(">")[1].split(" ")[0]
                species_split = line.split("[")
                if len(species_split) > 1:
                    species_id = species_split[1].split("]")[0]
                else:
                    species_id = None
               

            elif hit_id and "Length=" in line:
                seq_length = int(line.split("=")[-1].strip())
            elif hit_id and "Expect =" in line:
                evalue = line.split("Expect = ")[1].split(",")[0].strip()
                score = line.split("Score = ")[1].split(" bits")[0].strip()
            elif hit_id and "Identities =" in line:
                identities_str = line.split("Identities = ")[1].split(",")[0].strip()
                identities = identities_str.split(" ")[0]
                identities_fraction = identities.split("/")
                identities_percent = (int(identities_fraction[0]) / int(identities_fraction[1])) * 100
                positives_str = line.split("Positives = ")[1].split(",")[0].strip()
                positives = positives_str.split(" ")[0]
                positives_fraction = positives.split("/")
                positives_percent = (int(positives_fraction[0]) / int(positives_fraction[1])) * 100
                gaps_str = line.split("Gaps = ")[1].split(" ")[0].strip()
                gaps = gaps_str
                gaps_fraction = gaps.split("/")
                gaps_percent = (int(gaps_fraction[0]) / int(gaps_fraction[1])) * 100
                # Append all values to hits_data
                hits_data.append([
                    query_id, hit_id, hit_number, seq_length, species_id, evalue, score, 
                    identities, f"{identities_percent:.2f}%", positives, f"{positives_percent:.2f}%", gaps, f"{gaps_percent:.2f}%"
                ])
    return hits_data

# List to store parsed data
parsed_data = []

# Loop through all files in the source directory
for file_name in os.listdir(source_dir):
    if file_name.endswith("_blastp_results.txt"): 
        file_path = os.path.join(source_dir, file_name)
        parsed_data.extend(parse_blast_results(file_path))

# Write parsed data to CSV file
with open(output_csv, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # Write header
    writer.writerow([
        'Query ID', 'Hit ID', 'Hit Number', 'Sequence Length', 'Species ID', 'Evalue', 'Score', 
        'Identities', 'Identities (%)', 'Positives', 'Positives (%)', 'Gaps', 'Gaps (%)'
    ])
    # Write parsed data
    writer.writerows(parsed_data)

print("CSV file created successfully")


CSV file created successfully


### Move Fasta's back

In [30]:
import os
import shutil

# Path to the directory containing the fasta files
fasta_directory = "/home/osheakes/Research_Project_MMM/Fasta/"

# Destination directory to move the fasta files
destination_directory = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/matching_fasta_files/"

# Create the destination directory if it doesn't exist
os.makedirs(destination_directory, exist_ok=True)

# Counter for the number of files moved
files_moved_count = 0

# Iterate through each fasta file in the directory
for fasta_filename in os.listdir(fasta_directory):
    # Check if the file is a fasta file
    if fasta_filename.endswith(".fasta"):
        # Extract accession ID from the filename
        accession_id = fasta_filename.split("_")[0]
        
        # Check if the accession ID is in the blast results
        if accession_id in query_hits:
            # Path to the fasta file
            fasta_filepath = os.path.join(fasta_directory, fasta_filename)
            
            # Move the fasta file to the destination directory
            shutil.move(fasta_filepath, destination_directory)
            
            # Increment the count of files moved
            files_moved_count += 1
            
            print(f"Moved {fasta_filename} to {destination_directory}")

print(f"Finished moving {files_moved_count} fasta files.")


Finished moving 0 fasta files.


# 

In [10]:
import os

# Paths
no_hits_directory = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/no_hits"
blast_output_directory = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/nr_blastp_results"
blast_executable = "/home/osheakes/Research_Project_MMM/Fasta/ncbi-blast-2.15.0+/bin/tblastn_vdb"
db = "viruses_wgs"

# Check if the no_hits directory exists
if not os.path.isdir(no_hits_directory):
    raise FileNotFoundError(f"Directory not found: {no_hits_directory}")
else:
    print(f"Directory found: {no_hits_directory}")

# Check if the blast_output directory exists (if not, create it)
if not os.path.isdir(blast_output_directory):
    os.makedirs(blast_output_directory)
    print(f"Directory created: {blast_output_directory}")
else:
    print(f"Directory found: {blast_output_directory}")

# Check if the BLAST executable exists and is executable
if not os.path.isfile(blast_executable):
    raise FileNotFoundError(f"BLAST executable not found: {blast_executable}")
elif not os.access(blast_executable, os.X_OK):
    raise PermissionError(f"BLAST executable is not executable: {blast_executable}")
else:
    print(f"BLAST executable found and is executable: {blast_executable}")

# Check if there are any .fasta files in the no_hits directory
fasta_files = [f for f in os.listdir(no_hits_directory) if f.endswith(".fasta")]
if not fasta_files:
    raise FileNotFoundError(f"No .fasta files found in: {no_hits_directory}")
else:
    print(f"Found {len(fasta_files)} .fasta file(s) in: {no_hits_directory}")

print("Environment and path checks completed successfully.")


Directory found: /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/no_hits
Directory found: /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/nr_blastp_results
BLAST executable found and is executable: /home/osheakes/Research_Project_MMM/Fasta/ncbi-blast-2.15.0+/bin/tblastn_vdb
Found 79 .fasta file(s) in: /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/no_hits
Environment and path checks completed successfully.


# 

## 1.5 Creation of Master Blast file 

In [24]:
import os

# Directory containing blast result files
blast_output_directory = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts"

# Name of the master file
master_filename = "viral_master_blaster.txt"

# Path to the master file
master_filepath = os.path.join(blast_output_directory, master_filename)

# Open the master file in append mode
with open(master_filepath, "a") as master_file:
    # Iterate through each file in the blast output directory
    for filename in os.listdir(blast_output_directory):
        # Check if the file is a blast result file
        if filename.endswith("_blast_results.txt"):
            # Get the full path of the blast result file
            blast_result_filepath = os.path.join(blast_output_directory, filename)
            # Open the blast result file and read its content
            with open(blast_result_filepath, "r") as blast_result_file:
                # Read the content of the blast result file and write it to the master file
                master_file.write(blast_result_file.read())
                # Write a newline character to separate the content of each blast result file
                master_file.write("\n")

print("All blast_results.txt files have been concatenated into the master file:", master_filepath)


All blast_results.txt files have been concatenated into the master file: /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/viral_master_blaster.txt


In [19]:
import pandas as pd
import re

# Load the data from the CSV file
data = pd.read_csv("/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/fmt5/VB_Results.csv")

# Define search
pattern = r'\bhuman\b'

# Initialize count
human_count = 0

# Iterate over each row
for index, row in data.iterrows():
    if re.search(pattern, str(row['Species']), flags=re.IGNORECASE):
        human_count += 1

# Print the count
print(human_count)

# Filter rows
human_entries = data[data['Species'].str.contains('Human', case=False)]

# Save to CSV file
human_entries.to_csv("human_entries.csv", index=False)



616


## 1.6 Create Counter to identify which query had the most hits across the Viral database

In [20]:
from collections import Counter

# Path to the master file
master_filepath = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/viral_master_blaster.txt"

# Read the contents of the master file
with open(master_filepath, "r") as master_file:
    # Read all lines from the file
    lines = master_file.readlines()

# Extract query sequence IDs from each line
query_ids = [line.split()[0] for line in lines if line.strip()]

# Count the occurrences of each query ID
query_counts = Counter(query_ids)

# Sort the queries based on the number of hits (descending order)
ranked_queries = sorted(query_counts.items(), key=lambda x: x[1], reverse=True)

# Print the ranked list of queries
print("Ranked list of queries based on the number of hits:")
for rank, (query, count) in enumerate(ranked_queries, start=1):
    print(f"{rank}. Query ID: {query}, Hits: {count}")


Ranked list of queries based on the number of hits:
1. Query ID: sp|Q02388|CO7A1_HUMAN, Hits: 14725
2. Query ID: sp|P19838|NFKB1_HUMAN, Hits: 1757
3. Query ID: #, Hits: 1499
4. Query ID: sp|Q15303|ERBB4_HUMAN, Hits: 1487
5. Query ID: sp|P19525|E2AK2_HUMAN, Hits: 1435
6. Query ID: sp|Q00534|CDK6_HUMAN, Hits: 1165
7. Query ID: sp|P06493|CDK1_HUMAN, Hits: 1127
8. Query ID: sp|Q00526|CDK3_HUMAN, Hits: 1121
9. Query ID: sp|P24941|CDK2_HUMAN, Hits: 1063
10. Query ID: sp|P11802|CDK4_HUMAN, Hits: 1019
11. Query ID: sp|Q00535|CDK5_HUMAN, Hits: 1011
12. Query ID: sp|Q8NFJ6|PKR2_HUMAN, Hits: 1001
13. Query ID: sp|P32247|BRS3_HUMAN, Hits: 1001
14. Query ID: sp|P43490|NAMPT_HUMAN, Hits: 1001
15. Query ID: sp|P46663|BKRB1_HUMAN, Hits: 1001
16. Query ID: sp|P01019|ANGT_HUMAN, Hits: 1001
17. Query ID: sp|P55265|DSRAD_HUMAN, Hits: 919
18. Query ID: sp|Q9Y3Z3|SAMH1_HUMAN, Hits: 231
19. Query ID: sp|P47898|5HT5A_HUMAN, Hits: 141
20. Query ID: sp|P25445|TNR6_HUMAN, Hits: 141
21. Query ID: sp|P20591|MX1_HU

## 1.7 Cleaning up of dataset

In [4]:
# Filter out lines with query ID "#"
filtered_queries = [(query, count) for query, count in ranked_queries if query != "#"]

## 1.8 Create a Ranked table of the dataset

In [33]:
import csv

# Path to the output CSV file with the desired file name
output_csv_path = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/Ranked_Hits_Tab.csv"

# Write the filtered list of queries to a CSV file
with open(output_csv_path, "w", newline="") as csv_file:
    # Create a CSV writer object
    csv_writer = csv.writer(csv_file)
    
    # Write the header row without the "Rank" column
    csv_writer.writerow(["Query ID", "Hits"])
    
    # Write each ranked query to the CSV file without the rank
    for query, count in filtered_queries:
        csv_writer.writerow([query, count])

print("Ranked list of queries (excluding '#') saved to:", output_csv_path)


Ranked list of queries (excluding '#') saved to: /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/Ranked_Hits_Tab.csv


## 1.9 Create a filtered Viral Masterfile

In [51]:
# Read query IDs from Ranked_Hits_Tab.csv
query_ids_from_csv = set()
with open("/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/Ranked_Hits_Tab.csv", "r") as csv_file:
    next(csv_file)  # Skip header row
    for line in csv_file:
        query_id = line.strip().split(",")[0]
        query_ids_from_csv.add(query_id)

# Create a new .txt file to store filtered results
output_txt_path = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/filtered_viral_master_blaster.txt"
with open("/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/viral_master_blaster.txt", "r") as input_file, open(output_txt_path, "w") as output_file:
    for line in input_file:
        query_id = line.strip().split("\t")[0]
        if query_id in query_ids_from_csv:
            output_file.write(line)

print(f"Filtered results saved to {output_txt_path}")


Filtered results saved to /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/filtered_viral_master_blaster.txt


## 1.10 Create a Ranked Table for Sequence Homology

In [5]:
import csv

# Read the lines from the file
with open("/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/filtered_viral_master_blaster.txt", "r") as input_file:
    lines = input_file.readlines()

# Sort the lines based on the third column (sequence homology score) from highest to lowest
sorted_lines = sorted(lines, key=lambda x: float(x.strip().split("\t")[2]), reverse=True)

# Write the sorted lines to a CSV file with the header row
output_file_path = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/Ranked_Homology_Hits.csv"
with open(output_file_path, "w", newline="") as output_file:
    csv_writer = csv.writer(output_file)
    
    # Write the header row
    csv_writer.writerow(["query acc.ver", "subject acc.ver", "% identity", "alignment length", "mismatches", "gap opens", "q. start", "q. end", "s. start", "s. end", "evalue", "bit score"])
    
    # Write the sorted lines to the CSV file
    for line in sorted_lines:
        csv_writer.writerow(line.strip().split("\t"))

print(f"Ranked hits saved to: {output_file_path}")


Ranked hits saved to: /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/Ranked_Homology_Hits.csv


## 1.11 Retrieve Sequencing Data from Queries which Returned Hits

In [7]:
!mkdir  /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/processed_viral_seq

In [14]:
import pandas as pd

# Path to the file
file_path = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/Ranked_Homology_Hits.csv"

# Read the file into a DataFrame
df = pd.read_csv(file_path, delimiter="\t")  # Adjust delimiter if necessary

# Count the unique Accession IDs
unique_accession_ids = df['Accession ID'].nunique()

# Print the number of unique Accession IDs
print(f"Number of unique Accession IDs: {unique_accession_ids}")

KeyError: 'Accession ID'

# 

# 

# 

# 

# 

# 2.0 Bacterial Blast

In [64]:
!mkdir Bacterial_Blasts

In [1]:
!blastp -db nr -query P01225.fasta -out P01225_TEST.txt -evalue 0.06 -remote -entrez_query "bacteria[orgn]"

In [ ]:
import subprocess
import os

# Path to the directory containing fasta files
fasta_directory = "/home/osheakes/Research_Project_MMM/Fasta"

# Directory to store blast results
blast_output_directory = "/home/osheakes/Research_Project_MMM/Fasta/Bacterial_Blasts"

# Create the blast output directory if it doesn't exist
os.makedirs(blast_output_directory, exist_ok=True)

# Iterate through each fasta file in the directory
for filename in os.listdir(fasta_directory):
    if filename.endswith(".fasta"):  
        fasta_file = os.path.join(fasta_directory, filename)
        output_file = os.path.join(blast_output_directory, f"{os.path.splitext(filename)[0]}_blastp_results.txt")
        
        # Skip if output file already exists
        if os.path.exists(output_file):
            print(f"Skipping {filename} as output file already exists.")
            continue
        
        # Construct the blastp command
        blastp_command = [
            "blastp",
            "-db",
            "nr",
            "-query",
            fasta_file,
            "-out",
            output_file,
            "-evalue",
            "0.06",
            "-remote",
            "-entrez_query",
            "bacteria[orgn]"
        ]
        
        # Execute the blastp command
        subprocess.run(blastp_command)
        

print("Blastp searches completed for all fasta files.")


Skipping O96011.fasta as output file already exists.
Skipping Q07812.fasta as output file already exists.
Skipping Q8NFJ6.fasta as output file already exists.
Skipping P07492.fasta as output file already exists.
Skipping Q15303.fasta as output file already exists.
Skipping Q15726.fasta as output file already exists.
Skipping O00910.fasta as output file already exists.
Skipping P38398.fasta as output file already exists.
Skipping P06881.fasta as output file already exists.
Skipping P46663.fasta as output file already exists.
Skipping P32247.fasta as output file already exists.
Skipping P60484.fasta as output file already exists.
Skipping P20366.fasta as output file already exists.
Skipping P35318.fasta as output file already exists.
Skipping P12872.fasta as output file already exists.
Skipping P10645.fasta as output file already exists.
Skipping O95760.fasta as output file already exists.
Skipping P01344.fasta as output file already exists.
Skipping O14879.fasta as output file already e

### 

### 

### 

# 

# 

# 

# 

# 

# 3.0 Archaeal Blast

In [ ]:
!mkdir Archaeal_Blasts

## 3.1 Run Archaeal Blast 

In [2]:
import subprocess
import os
import time
from datetime import datetime, timezone

# Path to the directory containing fasta files
fasta_directory = "/home/osheakes/Research_Project_MMM/Fasta"

# Directory to store blast results
blast_output_directory = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts"

# Create the blast output directory if it doesn't exist
os.makedirs(blast_output_directory, exist_ok=True)

# Record the start time of the entire process in GMT
start_time = datetime.now(timezone.utc)
print(f"Process started at: {start_time.strftime('%Y-%m-%d %H:%M:%S')} GMT")

# Iterate through each fasta file in the directory
for filename in os.listdir(fasta_directory):
    if filename.endswith(".fasta"):  
        fasta_file = os.path.join(fasta_directory, filename)
        output_file = os.path.join(blast_output_directory, f"{os.path.splitext(filename)[0]}_blastp_results.txt")
        
        # Skip if output file already exists
        if os.path.exists(output_file):
            print(f"Skipping {filename} as output file already exists.")
            continue
        
        # Construct the blastp command
        blastp_command = [
            "blastp",
            "-db",
            "nr",
            "-query",
            fasta_file,
            "-out",
            output_file,
            "-evalue",
            "0.06",
            "-remote",
            "-entrez_query",
            "archaea[orgn]"
        ]
        
        # Record start time for this file
        file_start_time = time.time()
        
        # Execute the blastp command
        print(f"Running blast search for {filename}...")
        subprocess.run(blastp_command)
        
        # Calculate elapsed time for this file
        file_elapsed_time = time.time() - file_start_time
        
        # Print filename and elapsed time
        print(f"Blast search for {filename} completed in {file_elapsed_time:.2f} seconds.")
        
        # Check if 30 minutes have elapsed since the start of the process
        if (datetime.now(timezone.utc) - start_time).seconds >= 1800:
            print(f"Process has been running for 30 minutes. Current time: {datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S')} GMT")
            start_time = datetime.now(timezone.utc)  

print("Blastp searches completed for all fasta files.")



Process started at: 2024-05-02 20:59:15 GMT
Running blast search for O96011.fasta...
Blast search for O96011.fasta completed in 18161.19 seconds.
Process has been running for 30 minutes. Current time: 2024-05-03 02:01:56 GMT
Running blast search for Q07812.fasta...
Blast search for Q07812.fasta completed in 4905.02 seconds.
Process has been running for 30 minutes. Current time: 2024-05-03 03:23:41 GMT
Running blast search for Q8NFJ6.fasta...
Blast search for Q8NFJ6.fasta completed in 7622.79 seconds.
Process has been running for 30 minutes. Current time: 2024-05-03 05:30:44 GMT
Running blast search for P07492.fasta...
Blast search for P07492.fasta completed in 7013.17 seconds.
Process has been running for 30 minutes. Current time: 2024-05-03 07:27:37 GMT
Running blast search for Q15303.fasta...
Blast search for Q15303.fasta completed in 7326.64 seconds.
Process has been running for 30 minutes. Current time: 2024-05-03 09:29:44 GMT
Running blast search for Q15726.fasta...
Blast search f

## 3.2 Create Archaeal Master File

In [9]:
import os

# Specify Directory containing blast results
blast_output_directory = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts"

# Name the file
master_filename = "archaeal_master_blaster.txt"

# create Path to the master file
master_filepath = os.path.join(blast_output_directory, master_filename)

# Open the master file in append mode
with open(master_filepath, "a") as master_file:
    # Iterate through each file in the blast output directory
    for filename in os.listdir(blast_output_directory):
        # Check if the file is a blast result file
        if filename.endswith("_blast_results.txt"):
            # Get the path of the blast result file
            blast_result_filepath = os.path.join(blast_output_directory, filename)
            # Open the blast result file
            with open(blast_result_filepath, "r") as blast_result_file:
                # Write contents to master file
                master_file.write(blast_result_file.read())
                # Separate the content of each blast result file
                master_file.write("\n")

print("All blast_results.txt files have been concatenated into the master file:", master_filepath)


All blast_results.txt files have been concatenated into the master file: /home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/archaeal_master_blaster.txt


In [3]:
from collections import defaultdict
import csv

# Path to the master file
master_filepath = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/archaeal_master_blaster.txt"

# Initialize a defaultdict to store query counts and hits
query_hits = defaultdict(list)

# Read the file and extract query IDs and hits
try:
    with open(master_filepath, "r") as master_file:
        current_query_id = None
        for line in master_file:
            if "|" in line: 
                current_query_id = line.split("|")[1].strip()  
            elif ">" in line: 
                if current_query_id:
                    query_hits[current_query_id].append(line.split(">")[1].strip())  
except FileNotFoundError:
    print(f"Error: File '{master_filepath}' not found.")
    exit(1)
except Exception as e:
    print(f"An error occurred: {e}")
    exit(1)

# Calculate total hits for each accession ID
total_hits = {query_id: len(hits) for query_id, hits in query_hits.items()}

# Rank and print total hits
print("Ranked total hits for each accession ID:")
for rank, (query_id, hits) in enumerate(sorted(total_hits.items(), key=lambda x: x[1], reverse=True), start=1):
    print(f"{rank}. Accession ID: {query_id}, Total Hits: {hits}")

# Write results to CSV file
output_csv_path = " /home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/archaeal_master_blaster.csv"
try:
    with open(output_csv_path, mode='w', newline='') as csv_file:
        fieldnames = ['Query ID', 'Hits']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        
        writer.writeheader()
        for query_id, hits in query_hits.items():
            writer.writerow({'Query ID': query_id, 'Hits': ', '.join(hits)})
    print(f"Results saved to '{output_csv_path}'")
except Exception as e:
    print(f"An error occurred while writing to CSV: {e}")

Ranked total hits for each accession ID:
An error occurred while writing to CSV: [Errno 2] No such file or directory: ' /home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/archaeal_master_blaster.csv'


### 

### 

### 

# 

# 

# 

### 

## 4.0 Fungal Blast

In [4]:
!mkdir /home/osheakes/Research_Project_MMM/Fasta/Fungal_Blasts/

In [5]:
import subprocess
import os
import time
from datetime import datetime, timezone

# Path to the directory containing fasta files
fasta_directory = "/home/osheakes/Research_Project_MMM/Fasta"

# Directory to store blast results
blast_output_directory = "/home/osheakes/Research_Project_MMM/Fasta/Fungal_Blasts"

# Create the blast output directory if it doesn't exist
os.makedirs(blast_output_directory, exist_ok=True)

# Record the start time of the entire process in GMT
start_time = datetime.now(timezone.utc)
print(f"Process started at: {start_time.strftime('%Y-%m-%d %H:%M:%S')} GMT")

# Iterate through each fasta file in the directory
for filename in os.listdir(fasta_directory):
    if filename.endswith(".fasta"):  
        fasta_file = os.path.join(fasta_directory, filename)
        output_file = os.path.join(blast_output_directory, f"{os.path.splitext(filename)[0]}_blastp_results.txt")
        
        # Skip if output file already exists
        if os.path.exists(output_file):
            print(f"Skipping {filename} as output file already exists.")
            continue
        
        # Construct the blastp command
        blastp_command = [
            "blastp",
            "-db",
            "nr",
            "-query",
            fasta_file,
            "-out",
            output_file,
            "-evalue",
            "0.06",
            "-remote",
            "-entrez_query",
            "fungi[orgn]"
        ]
        
        # Record start time for this file
        file_start_time = time.time()
        
        # Execute the blastp command
        print(f"Running blast search for {filename}...")
        subprocess.run(blastp_command)
        
        # Calculate elapsed time for this file
        file_elapsed_time = time.time() - file_start_time
        
        # Print filename and elapsed time
        print(f"Blast search for {filename} completed in {file_elapsed_time:.2f} seconds.")
        
        # Check if 30 minutes have elapsed since the start of the process
        if (datetime.now(timezone.utc) - start_time).seconds >= 1800:
            print(f"Process has been running for 30 minutes. Current time: {datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S')} GMT")
            start_time = datetime.now(timezone.utc)  

print("Blastp searches completed for all fasta files.")


Process started at: 2024-05-05 02:31:31 GMT
Running blast search for O96011.fasta...
Blast search for O96011.fasta completed in 149.20 seconds.
Running blast search for Q07812.fasta...
Blast search for Q07812.fasta completed in 184.94 seconds.
Running blast search for Q8NFJ6.fasta...
Blast search for Q8NFJ6.fasta completed in 250.96 seconds.
Running blast search for P07492.fasta...
Blast search for P07492.fasta completed in 186.15 seconds.
Running blast search for Q15303.fasta...
Blast search for Q15303.fasta completed in 195.64 seconds.
Running blast search for Q15726.fasta...
Blast search for Q15726.fasta completed in 97.84 seconds.
Running blast search for O00910.fasta...
Blast search for O00910.fasta completed in 184.86 seconds.
Running blast search for P38398.fasta...
Blast search for P38398.fasta completed in 766.30 seconds.
Process has been running for 30 minutes. Current time: 2024-05-05 03:05:07 GMT
Running blast search for P06881.fasta...
Blast search for P06881.fasta complet

## 4.2 Create Fungal Master File 

In [8]:
import os

# Specify Directory containing blast results
blast_output_directory = "/home/osheakes/Research_Project_MMM/Fasta/Fungal_Blasts"

# Name the file
master_filename = "Fungal_master_blaster.txt"

# create Path to the master file
master_filepath = os.path.join(blast_output_directory, master_filename)

# Open the master file in append mode
with open(master_filepath, "a") as master_file:
    # Iterate through each file in the blast output directory
    for filename in os.listdir(blast_output_directory):
        # Check if the file is a blast result file
        if filename.endswith("_blast_results.txt"):
            # Get the path of the blast result file
            blast_result_filepath = os.path.join(blast_output_directory, filename)
            # Open the blast result file
            with open(blast_result_filepath, "r") as blast_result_file:
                # Write contents to master file
                master_file.write(blast_result_file.read())
                # Separate the content of each blast result file
                master_file.write("\n")

print("All blast_results.txt files have been concatenated into the master file:", master_filepath)


All blast_results.txt files have been concatenated into the master file: /home/osheakes/Research_Project_MMM/Fasta/Fungal_Blasts/Fungal_master_blaster.txt
